<a href="https://colab.research.google.com/github/Varvara225/NLP_hse_ling/blob/main/%D0%92%D0%92%D0%B0%D1%81%D0%B8%D0%BB%D1%8C%D0%B5%D0%B2%D0%B0_ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytube youtube-transcript-api SpeechRecognition pydub
!pip install git+https://github.com/openai/whisper.git soundfile
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
import speech_recognition as sr
from pydub import AudioSegment

In [ ]:
!pip install git+https://github.com/openai/whisper.git soundfile

In [ ]:
from transformers import pipeline
!pip install jiwer
import jiwer
from jiwer import wer

In [20]:
yt = YouTube('https://www.youtube.com/watch?v=clpV4BuX9EA')

In [ ]:
yt.streams.filter(only_audio=True).all()[0].download()

In [50]:
subs = YouTubeTranscriptApi.get_transcript('clpV4BuX9EA', languages=['en'])
yt_text = ''

In [62]:
# тестировать буду первые 100 секунд видео
for s in subs:
  if s['start'] == 103.74:
    break
  yt_text +=  s['text']
  yt_text += ' '

yt_text

"I ran in this other issue in in France and it's uh okay it's a hideous story before I get into it it's about cheeseburgers so when I order cheeseburgers I just want bun cheese and meat is anyone else the same way I know okay we're very rare that people hate us for it but like every time the waiter comes to I'm like I just want cheese and like just cheese and like yes just cheese I don't understand lettuce as nothing to the conversation it's crunchy water and also I hate pickles I hey oh I oh pickles are nothing but cucumbers sitting in their own piss that's what a pickle is yes and if you ask for no pickles seven pickles will show up on your plate but I always ask boys I'm like just cheese just cheese just cheese they always make a fuss about it then when it comes your friends are like what are you a seventh grader I'm like you eat ass Shut the up let me eat my Burger so I always want just cheese so okay so I'm in Paris right and we're we're drunk and it's late and the only place open

In [67]:
f = open("YouTube_subs.txt", "a")
f.write(yt_text)
f.close()

Далее я изменила формат аудио (wav) и переименовала более кратким названием. Далее пришлось также решить ошибку касательно стерео и моно записи.

In [ ]:
!pip install ffmpeg
!ffmpeg -i "Matteo Lane - Ordering Cheeseburgers In France.mp4" -ac 2 -f wav "burgers.wav"

In [ ]:
stereo_audio = AudioSegment.from_file(
    "burgers.wav",
    format="wav")

mono_audios = stereo_audio.split_to_mono()

mono_left = mono_audios[0].export(format="wav")
mono_right = mono_audios[1].export(format="wav")

In [ ]:
mono_left = mono_audios[0].export('left.wav', format="wav")
mono_right = mono_audios[1].export('right.wav', format="wav")

После приведения к монозаписи, каждая часть по сути осталась такой же длины, как изначальное аудио, но зато пропала ошибка. Я решила взать запись right и обрезала аудио до около 100 секунд.

 Я пользовалась библиотекой SpeechRecognition и transformers.

In [54]:
r = sr.Recognizer()
with sr.AudioFile('right_cut.wav') as source:
    audio = r.record(source)

In [55]:
# Система ASR whisper
whisper = r.recognize_whisper(audio)

In [25]:
# Система ASR google
google = r.recognize_google(audio)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
# Система ASR wav2vec2
transcriber = pipeline("automatic-speech-recognition", "facebook/wav2vec2-base-960h")

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [28]:
wav2vec = transcriber('right_cut.wav')

Из автоматических субтитров YouTube я также вязяла соовтетвующий 100 первым секундам пассаж текста. В качестве эталона я взяла расшифровку whisper и отредактировала.

In [57]:
gold = """I ran in this other issue in France and it's uh okay it's a hideous story. Before I get into it, it's about cheeseburgers. So when I order cheeseburger, I just want bun cheese and meet. Is anyone else the same way? I know. OK. We're very rare that people hate us for it. But every time the waiter comes in, I'm like, I just want cheese. And I'm like, just cheese. And yes, just cheese. I don't understand lettuce as nothing to the conversation is crunchy water. And also, I hate pickles. I fucking hate. Oh, I. Oh. Pickles are nothing but cucumbers sitting in their own piss. That's what a pickle is. Yes. And if you ask for no pickles, seven pickles will show up on your plate. But I always ask ways of like, just cheese, just cheese, just cheese. They always make a fuss about it. Then when it comes, your friends will like, what are you with? Seventh grader? I'm like, you eat ass. Shut the fuck up. Let me eat my fucking burger. So I always want just cheese. OK. So I'm in Paris, right? And we're drunk and it's late. And the only place open is McDonald's. And I was like, let's get McEdie's. So we're standing in line. And I asked my friend, my friend, the guy I'm sleeping with. And I'm like, how do I say just cheese and French? And he's like, solement de fromage. I'm like, seulement de fromage. He's like, solement de fromage. I'm like, OK. So I walk up to the woman working there. And she's like, mic donnell, bonsoir. And I was like, bonsoir. Je voudrais en cheese booger avec seulement de fromage. She stopped typing. She looks at me and she goes, solement de fromage. I'm like, oh my god, it's happening in another country right now."""

In [33]:
transf = jiwer.Compose(
    [
        jiwer.ExpandCommonEnglishContractions(),
        jiwer.RemoveEmptyStrings(),
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords(),
    ]
)

In [37]:
wer = jiwer.wer(
                gold,
                whisper,
                truth_transform=transf,
                hypothesis_transform=transf,
            )
print(f"Word Error Rate (WER) of Whisper :", wer)

Word Error Rate (WER) of Whisper : 0.03283582089552239


In [36]:
wer = jiwer.wer(
                gold,
                google,
                truth_transform=transf,
                hypothesis_transform=transf,
            )
print(f"Word Error Rate (WER) of Google :", wer)

Word Error Rate (WER) of Google : 0.6567164179104478


In [65]:
wer = jiwer.wer(
                gold,
                wav2vec['text'],
                truth_transform=transf,
                hypothesis_transform=transf,
            )
print(f"Word Error Rate (WER) of Wav2vec :", wer)

Word Error Rate (WER) of Wav2vec : 0.49258160237388726


In [66]:
wer = jiwer.wer(
                gold,
                yt_text,
                truth_transform=transf,
                hypothesis_transform=transf,
            )
print(f"Word Error Rate (WER) of YouTube auto subs :", wer)

Word Error Rate (WER) of YouTube auto subs : 0.17804154302670624
